# Install and dependencies

In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa
import librosa
import numpy as np
import math
import json

# Load data

In [ ]:
# df = pd.read_csv("../input/alldatapicklebirdclef2022/5_second_clips.csv") 
df = pd.read_csv("../input/alldatapicklebirdclef2022/5_second_clips.csv") 
df = df.drop(columns=["Unnamed: 0"])
df = df.sample(frac=1).reset_index(drop=True)

df

In [ ]:
df_train = df[0:130000]
df_test = df[130000:]
df_test = df_test.reset_index(drop=True)

In [ ]:
df_stat = df_train.groupby("bird_type").count().reset_index()
df_stat = df_stat.rename(columns={"file_path": "n_files"})
df_stat

# Oversample the train dataset

In [ ]:
target_duration = 10623
df_train_new = df_train.copy(deep=True)
bird_types = list(df_stat["bird_type"])
for i, bird_type in enumerate(bird_types):
    # print(f"Appending bird type {bird_type} ... {i}/152")
    # Get the data frame with the specific bird_type
    df_bird = df_train[df_train["bird_type"]==bird_type]
    songs = list(df_bird["file_path"]) * 15000
    # Get the current total duration in seconds
    n_files = int(df_stat[df_stat["bird_type"]==bird_type]["n_files"])
    
    take_n_files = target_duration - n_files
    
    song_appended = songs[:take_n_files]
    bird_type_appended = [bird_type] * take_n_files
        
    df_tmp = pd.DataFrame({"bird_type": bird_type_appended,
                           "file_path": song_appended})
    df_train_new = pd.concat([df_train_new, df_tmp])
df_train_new

In [ ]:
# root_file = os.path.abspath("../input/alldatapicklebirdclef2022/timesum/timesum")
root_file = os.path.abspath("../input/alldatapicklebirdclef2022/timesum/timesum")
df_train_new["files_timesum"] = df_train_new.apply(lambda x: os.path.join(root_file, x["file_path"]), axis=1)
# root_file = os.path.abspath("../input/alldatapicklebirdclef2022/freqsum/freqsum")
root_file = os.path.abspath("../input/alldatapicklebirdclef2022/freqsum/freqsum")
df_train_new["files_freqsum"] = df_train_new.apply(lambda x: os.path.join(root_file, x["file_path"]), axis=1)
df_train_new

In [ ]:
# root_file = os.path.abspath("../input/alldatapicklebirdclef2022/timesum/timesum")
root_file = os.path.abspath("../input/alldatapicklebirdclef2022/timesum/timesum")
df_test["files_timesum"] = df_test.apply(lambda x: os.path.join(root_file, x["file_path"]), axis=1)
# root_file = os.path.abspath("../input/alldatapicklebirdclef2022/freqsum/freqsum")
root_file = os.path.abspath("../input/alldatapicklebirdclef2022/freqsum/freqsum")
df_test["files_freqsum"] = df_test.apply(lambda x: os.path.join(root_file, x["file_path"]), axis=1)
df_test

In [ ]:
df_train_new = df_train_new.sample(frac=1)

In [ ]:
df_train_new["files_timesum"][0], df_train_new["files_freqsum"][0]

# Asign index to each bird class

In [ ]:
bird_types = set(list(df["bird_type"])) 
bird_type_dict = dict(zip(bird_types, np.arange(len(bird_types))))
df_train_new["bird_type_index"] = df_train_new.apply(lambda x: bird_type_dict[x["bird_type"]], axis=1)
df_test["bird_type_index"] = df_test.apply(lambda x: bird_type_dict[x["bird_type"]], axis=1)

# Build Dataloading Function

In [ ]:
def preprocess(file_path, label): 
    npy = np.load(file_path.decode('utf-8'))
    spectrogram = tf.convert_to_tensor(npy)
    spectrogram = tf.expand_dims(spectrogram, axis=1)
    #spectrogram = tf.io.read_file(file_path)
    # label = tf.one_hot(label, depth=152)
    label = tf.convert_to_tensor(label, dtype=tf.int32)
    return spectrogram, label

In [ ]:

files = list(df_train_new["files_timesum"])
labels = list(df_train_new["bird_type_index"])
data_timesum = tf.data.Dataset.from_tensor_slices((files, labels))

data_timesum = data_timesum.map(lambda item1, item2: tf.numpy_function(
        preprocess, [item1, item2], [tf.float32, tf.int32]),
        num_parallel_calls=tf.data.AUTOTUNE)

# data_timesum = data_timesum.cache()
data_timesum = data_timesum.shuffle(buffer_size=1000)
data_timesum = data_timesum.batch(16)
data_timesum_train = data_timesum.prefetch(8)

# data_timesum.as_numpy_iterator().next() 


In [ ]:

files = list(df_test["files_timesum"])
labels = list(df_test["bird_type_index"])
data_timesum = tf.data.Dataset.from_tensor_slices((files, labels))

data_timesum = data_timesum.map(lambda item1, item2: tf.numpy_function(
        preprocess, [item1, item2], [tf.float32, tf.int32]),
        num_parallel_calls=tf.data.AUTOTUNE)

# data_timesum = data_timesum.cache()
data_timesum = data_timesum.shuffle(buffer_size=1000)
data_timesum = data_timesum.batch(16)
data_timesum_test = data_timesum.prefetch(8).repeat()

# data_timesum.as_numpy_iterator().next() 


In [ ]:
s, l = data_timesum_train.as_numpy_iterator().next()
s.shape, l.shape

# Merge two datasets

In [ ]:
train = data_timesum_train
test = data_timesum_test

In [ ]:
s, l = train.as_numpy_iterator().next()
s.shape, l.shape

# Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, Flatten, MaxPool1D, MaxPool2D, Dropout, BatchNormalization

In [ ]:
model = Sequential([
    
    Conv1D(64, 3, activation='relu', input_shape=(513,1)),
    Dropout(0.5),
    Conv1D(64, 3, activation='relu'),
    Dropout(0.5),
    Conv1D(64, 3, activation='relu'),
    Dropout(0.5),
    MaxPool1D(2),
    Flatten(),
    Dense(1024, activation='relu'), # You have 152 bird types, check this with len(bird_types)
    Dense(152, activation='softmax'), # You have 152 bird types, check this with len(bird_types)
])
model.summary()

In [ ]:
x = tf.ones((16, 513, 1))
y=model(x)
y.shape

In [ ]:
model.compile(optimizer="Adam", 
              loss= "sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy" ]
              )
model.evaluate(test, steps=64)

# Learning Rate scheduler

In [ ]:
class DetectHighAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold=0.98):
        super(DetectHighAccuracyCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs["val_sparse_categorical_accuracy"] > self.threshold:
            print(f"Stopping training, reached {self.threshold} accuracy. ")
            self.model.stop_training = True

In [ ]:
train_dir_ckpt = "./training_version_18"
if os.path.exists(train_dir_ckpt) == False:
    os.mkdir(train_dir_ckpt)

In [ ]:
checkpoint_path = f"{train_dir_ckpt}/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#https://www.tensorflow.org/tutorials/keras/save_and_load
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-5 if epoch < 5 else 4.E-04)

hist = model.fit(train, 
                 epochs=100, 
                 validation_data=test, 
                 steps_per_epoch=512,
                 validation_steps=64,
#                 callbacks=[lr_schedule, 
#                           DetectHighAccuracyCallback(),
#                           cp_callback
#                           ],
                 verbose=1)

In [ ]:
plt.clf()
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="validation")
plt.savefig(f"./{train_dir_ckpt}/loss.png")
plt.savefig(f"./{train_dir_ckpt}/loss.pdf")

In [ ]:
plt.clf()
plt.plot(hist.history["sparse_categorical_accuracy"], label="test")
plt.plot(hist.history["val_sparse_categorical_accuracy"], label="validation")
plt.savefig(f"./{train_dir_ckpt}/sparse_categorical_accuracy.png")
plt.savefig(f"./{train_dir_ckpt}/sparse_categorical_accuracy.pdf")


In [ ]:
# hist.history

In [ ]:
max(hist.history["val_sparse_categorical_accuracy"])

# Submit the solution

In [ ]:
bird_type_dict_inverse = {v: k for k, v in bird_type_dict.items()}
# bird_type_dict_inverse

In [ ]:
# "aniani" in list(bird_type_dict_inverse.values())

In [ ]:
def load_wav_16k_mono(file_input, offset):
    # Args: 
    # file_name: path to the .ogg file
    # total_duration: how much the song lasts
    # Output: you will take only 5 second of each song.
    
    wav, sample_rate = librosa.load(file_input, 
                                    sr=16000, 
                                    duration=5,
                                    offset=offset)
    return wav


def load_spectrogram(file_input, offset):
    wav = load_wav_16k_mono(file_input, offset)
    # 16000 data = 1 seconds
    num_data = int(16000 * 5)
    frame_length=1024
    frame_step=128
    wav = wav[:num_data]
    zero_padding = tf.zeros([num_data] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=frame_length, frame_step=frame_step)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram / tf.reduce_max(spectrogram)
    return spectrogram.numpy()
    

def preprocess(file_input, offset):
    spectrogram = load_spectrogram(file_input, offset)
    spectrogram = np.sum(spectrogram, axis=0)
    spectrogram *= 1. / np.max(spectrogram)
    spectrogram = np.expand_dims(spectrogram, axis=1)
    return spectrogram
    
    
def predict_label(model, file_id, offset):
    #file_input = os.path.join("../input/birdclef-2022/test_soundscapes/", f"{file_id}")
    file_input = file_id
    x = preprocess(file_input, offset)
    x = tf.expand_dims(x, axis=0)
    prediction = model(x)
    prediction_label = tf.argmax(prediction, axis=1).numpy()[0]
    return prediction_label
    
def predict_label_npy(model, file_id, offset):
    # file_input = os.path.join("../input/birdclef-2022/test_soundscapes/", f"{file_id}")
    #file_input = os.path.join("../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg")
    file_input = file_id
    
    
    x = np.load(file_input)
    x = tf.expand_dims(x, axis=0)
    x = tf.expand_dims(x, axis=2)
    prediction = model(x)
    prediction_label = tf.argmax(prediction, axis=1).numpy()[0]
    return prediction_label
    
def predict(model, file_id, offset):
    prediction_label = predict_label(model, file_id, offset)
    predicted_bird = bird_type_dict_inverse[prediction_label]
    return predicted_bird
    

In [ ]:
df_train["file_path"][2]

In [ ]:
file_input = "../input/birdclef-2022/train_audio/arcter/XC432747.ogg"
x = preprocess(file_input, offset=0)
x = tf.expand_dims(x, axis=0)
prediction = predict(model, file_id=file_input, offset=9*5)
prediction

In [ ]:
file_input = "../input/birdclef-2022/train_audio/bcnher/XC636465.ogg"
x = preprocess(file_input, offset=0)
x = tf.expand_dims(x, axis=0)
prediction = predict(model, file_id=file_input, offset=3*5)
prediction

In [ ]:
n_tot = 100
n_correct = 0
for i, file_path in enumerate(list(df_test["file_path"])):
    if i>n_tot:
        break
    offset = int(file_path.split("_")[-1].split(".")[0])
    file_input = file_path.replace("_" + str(offset), "").replace("npy", "ogg")
    bird = file_path.split("/")[0]
    file_input = f"../input/birdclef-2022/train_audio/{file_input}"
    x = preprocess(file_input, offset=0)
    x = tf.expand_dims(x, axis=0)
    prediction = predict(model, file_id=file_input, offset=offset*5)
    n_correct += 1 if prediction == bird else 0
    print(f"Real bird: {bird}. Predicted: {prediction}")
print(f"Correctly guessed: {n_correct}. Percentage of correct: {n_correct / n_tot}")

In [ ]:
y_true = list(df_test["bird_type_index"])[0:1000]
files = list(df_test["files_timesum"])[0:1000]
y_pred = [predict_label_npy(model, file, offset=0) for file in files]

In [ ]:
import sklearn
f1_score = sklearn.metrics.f1_score(y_true, y_pred, average="macro")
f1_score

In [ ]:
from sklearn.metrics import confusion_matrix
#Get the confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(30,20))


ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_xlabel('\nPredicted')
ax.set_ylabel('Actual ');

## Ticket labels - List must be in alphabetical order
#ax.xaxis.set_ticklabels(bird_names, rotation=45)
#ax.yaxis.set_ticklabels(bird_names, rotation=45)

plt.show()

In [ ]:
np.sum(cf_matrix, axis=1) 

In [ ]:
np.diag(cf_matrix) # np.shape(cf_matrix)

In [ ]:
plt.hist(y_pred)

In [ ]:
input_dir="/kaggle/input/birdclef-2022"

pred={
  'row_id':[],
  'target':[]
}

test_path=input_dir+"/test_soundscapes/"
# files= list(df_test["file_id"]) # [f.split('.')[0] for f in sorted(os.listdir(test_path))]
files= [f.split('.')[0] for f in sorted(os.listdir(test_path))]
print(files)

birds_path=input_dir+"/scored_birds.json"
with open(birds_path) as bf:
    birds = json.load(bf)

for f in files:
    p=test_path+f+'.ogg'
    p = os.path.join("../input/birdclef-2022/test_soundscapes/", f"{p}")
    
    d=librosa.get_duration(filename=p)
    pcs=round(d/5)
    segments = [[] for i in range(pcs)]
      
    for i in range(len(segments)):
        offset = int(i * 5)
        segment_end=(i+1)*5  
        predicted_bird = predict(model, p, offset)
        for b in birds:
            print(f'Predicted bird: {predicted_bird} [{bird_type_dict[predicted_bird]}]. Real Bird: {b}')
            row_id=f+'_'+b+'_'+str(segment_end)
            pred['row_id'].append(row_id)
            is_prediction_true = (predicted_bird == b)
            pred['target'].append(is_prediction_true)

In [ ]:
cols=['row_id','target']
df_sub=pd.DataFrame({'row_id': pred['row_id'],
                   'target': pred['target']})
df_sub

In [ ]:
list(df_sub["row_id"])

In [ ]:
True in list(df_sub["target"])

In [ ]:
# test_df

In [ ]:
work_dir="/kaggle/working"       
df_sub.to_csv(work_dir+"/submission.csv", index=False)
df_sub.to_csv("submission.csv", index=False)


In [ ]:
work_dir+"/submission.csv"